# Example Code for Implement the Noise Model

## Basic Installation

Install required package, we highly recommend participant to use qiskit platform, or at least participants can finish preprocessing at other platform and transfer the circuit to qiskit format, since our noise model is from IBM real machine backend and we restricted some algorithmic seeds which could be varied from different platform.

# Implement Noise Model

In [ ]:
import numpy as np
from qiskit import QuantumCircuit, transpile
from qiskit.quantum_info import Kraus, SuperOp
from qiskit_aer import AerSimulator
from qiskit.tools.visualization import plot_histogram
from qiskit_aer.noise import (NoiseModel, QuantumError, ReadoutError,
    pauli_error, depolarizing_error, thermal_relaxation_error)
import numpy as np
# from qiskit.tools.jupyter import *
import warnings
warnings.filterwarnings('ignore')
from qiskit import *
import time
from qiskit.providers.aer.noise import NoiseModel
import qiskit.providers.aer.noise as noise
from qiskit.utils import QuantumInstance, algorithm_globals

In [ ]:
from qiskit.providers.fake_provider import *

In [ ]:
import pickle

In [ ]:
with open('NoiseModel/fakekolkata.pkl', 'rb') as file:
    noise_model = pickle.load(file)

In [ ]:
noise_model

In [ ]:
noise_model1 = noise.NoiseModel()

In [ ]:
noise_model1

In [ ]:
noise_modelreal = noise_model1.from_dict(noise_model)

In [ ]:
noise_modelreal

Transfer the noise model to Pennylane: https://pennylane.ai/blog/2021/05/how-to-simulate-noise-with-pennylane/

Here we require paticipants to fix the algorithm seed in qiskit. *MUST* translate other format circuit to qiskit before any place need algorithm seed. And we give 20, 21, 30, 33, 36, 42, 43, 55, 67, 170 as set of seeds to test results on your side, and the result will be calculated as the average of results from some of those seeds with some hidden seeds. And please use shots as 6000.

In [ ]:
seeds = 170
algorithm_globals.random_seed = seeds
seed_transpiler = seeds
shot = 6000

# Circuit Transpile and Execution

In [ ]:
from qiskit.circuit.random import random_circuit
from qiskit_aer.primitives import Estimator
circuit = random_circuit(2, 2, seed=0).decompose(reps=1)
circuit.draw()

Transpile the circuit based on given system model from IBMQ_Montreal.

In [ ]:
system_model = FakeMontreal()

In [ ]:
transpiled_circuit = transpile(circuit, backend=system_model)

In [ ]:
from qiskit.quantum_info import SparsePauliOp

observable = SparsePauliOp("IIIIIIIIIIIIIIIIIIIIIIIIIXZ")
print(f">>> Observable: {observable.paulis}")

Evaluate on the qiskit Estimator.

In [ ]:
estimator = Estimator(
    backend_options = {
        'method': 'statevector',
        'device': 'CPU',
        'noise_model': noise_modelreal
    },
    run_options = {
        'shots': shot,
        'seed': seeds,
    },
    skip_transpilation=True
)

In [ ]:
job = estimator.run(transpiled_circuit,observable)
result = job.result()
print(f">>> {result}")

## Obtain the Duration of Quantum Circuit

In [ ]:
from qiskit import pulse

Do remember to set the optimization_level to 0 if your circuit is already transpiled!!!

In [ ]:
with pulse.build(system_model) as my_program1:
  with pulse.transpiler_settings(optimization_level=0):
    pulse.call(transpiled_circuit)

In [ ]:
my_program1.duration

# Save the Circuit in QASM Format

In [ ]:
transpiled_circuit.qasm()